In [8]:
!pip install aiohttp


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import asyncio
import aiohttp
import time

async def send_request(session, url, data, headers):
    start_time = time.time()
    async with session.post(url, json=data, headers=headers) as response:
        await response.text()  # Wait for response
    end_time = time.time()
    return end_time - start_time  # Return the response time

async def send_requests_concurrently(rate, duration, total_requests):
    url = 'http://127.0.0.1:8000/inference'
    headers = {'accept': 'application/json', 'Content-Type': 'application/json'}
    data = {
        "img_path": "apple.jpeg",
        "camera_id": "string"
    }

    async with aiohttp.ClientSession() as session:
        tasks = []
        start_time = time.time()
        while len(tasks) < total_requests:
            if len(tasks) % rate == 0 and len(tasks) != 0:
                await asyncio.sleep(1)  # Wait for one second before sending the next batch

            if time.time() - start_time > duration:
                break  # Stop if the total duration has been exceeded

            task = asyncio.create_task(send_request(session, url, data, headers))
            tasks.append(task)

            await asyncio.sleep(1 / rate)  # Delay to control the rate of requests per second

        response_times = await asyncio.gather(*tasks)
        return response_times

# Function to run the async function from a running event loop
async def run_async_main():
    total_requests = 150
    rate_per_second = 30
    duration_seconds = 5  # Adjust the duration as needed
    response_times = await send_requests_concurrently(rate_per_second, duration_seconds, total_requests)

    # Calculate the average response time
    average_response_time = sum(response_times) / len(response_times)
    print(f"Average Response Time: {average_response_time:.4f} seconds")

# Execute the function using the existing event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(run_async_main())


RuntimeError: This event loop is already running

Image successfully downloaded: downloaded_image.jpg
